### 조선 왕조 실록 분석 : 시대별 라벨링하기

전제 : 데이터는 날짜별로 인덱싱이 가능한 상태라 판단

순서 - 크롤링 과정이 필요하므로 모델부터 제작
1. 라벨 설정 : 어디서부터 어디까지, 구체적인 날짜
2. 라벨링
3. 토크나이징
4. 임베딩
5. 모델링
6. 결과 확인

1. 라벨 설정

3. 토크나이징
   - 토크나이저 : Okt, Mecab, Kkma, Komoran, Hannanum
   - Okt : Open Korean Text; 트위터에서 만든 한국어 처리기; 장점 : 속도가 빠르다, 단점 : 정확도가 떨어진다
   - Mecab : 일본어 형태소 분석기; 한국어도 사용 가능; 장점 : 정확도가 높다, 단점 : 속도가 느리다
   - Kkma : 서울대학교에서 만든 한국어 처리기; 장점 : 정확도가 높다, 단점 : 속도가 느리다
   - Komoran : Shineware에서 만든 한국어 처리기; 장점 : 정확도가 높다, 단점 : 속도가 느리다
   - Hannanum : KAIST에서 만든 한국어 처리기; 장점 : 정확도가 높다, 단점 : 속도가 느리다

3-1. 토크나이저 테스트
- 실록 예문을 가져와 어떤 토크나이저가 좋을지 판단해보기

In [1]:
example = './example_words.txt'

def read_words(filename):
    with open(filename, 'r') as f:
        words = f.read().splitlines()
    return words

print(read_words(example))

['임금이 군신을 거느리고 인정전에서 하정례(賀正禮)를 행하고 백관의 조하(朝賀)를 정지하였다. 군신이 안장 갖춘 말을 드리고, 각도에서 전문(箋文)을 받들고 칭하(稱賀)가 끝나매, 방물(方物)을 바치고, 또 대수박(遞手帕)을 공비전(恭妃殿)에 드렸다. 임금이 이어소(移御所)에 나아가 백관을 거느리고 하례를 하고, 안팎 옷감[表裏]과 안장 갖춘 말을 드리고, 또 대수박을 대비전(大妃殿)에 올렸다. 임금이 이에 내전에 들어가 헌수(獻壽)하였다.', '대행 상왕의 재궁(梓宮)이 천수사(天壽寺)의 앞뜰에 머물렀다.']


In [16]:
# tokenize words
# 1. Okt
from konlpy.tag import Okt
import re
okt = Okt()
words = read_words(example)

tokens = []
for word in words:
    tokens.extend(okt.morphs(word))

def preprocess(tokens):
    # 정규표현식으로 한글만 추출
    tokens = [token for token in tokens if re.match(r'[가-힣]+', token)]
    # 불용어 제거
    stopwords = ['을', '를', '이', '가', '은', '는']
    tokens = [token for token in tokens if token not in stopwords]
    return tokens

# 15글자씩 끊어서 출력
tokens = preprocess(tokens)
for i in range(0, len(tokens), 15):
    print(tokens[i:i+15])

['임금', '군신', '거', '느리고', '인정전', '에서', '하', '정례', '행', '하고', '백관', '의', '조하', '정지', '하였다']
['군신', '안장', '갖춘', '말', '드리고', '각도', '에서', '전문', '받', '들고', '칭하', '끝나매', '방물', '바치', '고']
['또', '대', '수박', '공비', '전', '에', '드렸다', '임금', '어소', '에', '나아가', '백관', '거', '느리고', '하례']
['하고', '안팎', '옷감', '과', '안장', '갖춘', '말', '드리고', '또', '대', '수박', '대비', '전', '에', '올렸다']
['임금', '에', '내전', '에', '들어가', '헌수', '하였다', '대행', '상왕', '의', '재궁', '천', '수사', '의', '앞뜰']
['에', '머물렀다']


In [24]:
# 각 토크나이저 비교
# 1. Okt
tokens = []
for word in words:
    tokens.extend(okt.morphs(word))

tokens = preprocess(tokens)

print('================ Okt ================')
for i in range(0, len(tokens), 15):
    print(tokens[i:i+15])

# 2. Kkma
from konlpy.tag import Kkma
kkma = Kkma()

tokens = []
for word in words:
    tokens.extend(kkma.morphs(word))

tokens = preprocess(tokens)
print('================ Kkma ================')
for i in range(0, len(tokens), 15):
    print(tokens[i:i+15])
    
# 3. Hannanum
from konlpy.tag import Hannanum
hannanum = Hannanum()

tokens = []
for word in words:
    tokens.extend(hannanum.morphs(word))
    
tokens = preprocess(tokens)
print('================ Hannanum ================')

for i in range(0, len(tokens), 15):
    print(tokens[i:i+15])
    
# 4. Komoran
from konlpy.tag import Komoran
komoran = Komoran()

tokens = []
for word in words:
    tokens.extend(komoran.morphs(word))
    
tokens = preprocess(tokens)
print('================ Komoran ================')

for i in range(0, len(tokens), 15):
    print(tokens[i:i+15])

# # 5. Mecab
# from konlpy.tag import Mecab
# mecab = Mecab()

# tokens = []
# for word in words:
#     tokens.extend(mecab.morphs(word))
    
# tokens = preprocess(tokens)
# print('================ Mecab ================')

# for i in range(0, len(tokens), 15):
#     print(tokens[i:i+15])

================ Okt ================
['임금', '군신', '거', '느리고', '인정전', '에서', '하', '정례', '행', '하고', '백관', '의', '조하', '정지', '하였다']
['군신', '안장', '갖춘', '말', '드리고', '각도', '에서', '전문', '받', '들고', '칭하', '끝나매', '방물', '바치', '고']
['또', '대', '수박', '공비', '전', '에', '드렸다', '임금', '어소', '에', '나아가', '백관', '거', '느리고', '하례']
['하고', '안팎', '옷감', '과', '안장', '갖춘', '말', '드리고', '또', '대', '수박', '대비', '전', '에', '올렸다']
['임금', '에', '내전', '에', '들어가', '헌수', '하였다', '대행', '상왕', '의', '재궁', '천', '수사', '의', '앞뜰']
['에', '머물렀다']
================ Kkma ================
['임금', '군신', '거느리', '고', '인정전', '에서', '하정례', '행하', '고', '백관', '의', '조하', '정지', '하', '였']
['다', '군', '신이', '안장', '갖추', '말', '드리', '고', '각도', '에서', '전문', '받들', '고', '칭', '하']
['아', '끝나', '매', '방물', '바치', '고', '또', '대수', '박', '공', '비전', '에', '드리', '었', '다']
['임금', '잇', '어', '소', '에', '나', '아가', '백관', '거느리', '고', '하례', '하', '고', '안팎', '옷감']
['과', '안장', '갖추', '말', '드리', '고', '또', '대수', '박', '대비전', '에', '올리', '었', '다', '임금']
['이에', '내전', '에', '들어가', '헌수', '하', '었', '다

In [ ]:
# 일단 Okt가 가장 적합해 보임

In [ ]:
# Mecab을 맥에서 사용해보도록 만들기
from konlpy.tag import Mecab
mecab = Mecab()

words = read_words(example)

tokens = []
for word in words:
    tokens.extend(mecab.morphs(word))
    
tokens = preprocess(tokens)
for i in range(0, len(tokens), 15):
    print(tokens[i:i+15])

추가) 빈도수 출력해보기

In [26]:
# Okt 이용하여 단어 빈도수 계산
from collections import Counter

tokens = []
for word in words:
    tokens.extend(okt.morphs(word))

tokens = preprocess(tokens)
counter = Counter(tokens)

print(counter)

Counter({'에': 6, '임금': 3, '의': 3, '군신': 2, '거': 2, '느리고': 2, '에서': 2, '하고': 2, '백관': 2, '하였다': 2, '안장': 2, '갖춘': 2, '말': 2, '드리고': 2, '또': 2, '대': 2, '수박': 2, '전': 2, '인정전': 1, '하': 1, '정례': 1, '행': 1, '조하': 1, '정지': 1, '각도': 1, '전문': 1, '받': 1, '들고': 1, '칭하': 1, '끝나매': 1, '방물': 1, '바치': 1, '고': 1, '공비': 1, '드렸다': 1, '어소': 1, '나아가': 1, '하례': 1, '안팎': 1, '옷감': 1, '과': 1, '대비': 1, '올렸다': 1, '내전': 1, '들어가': 1, '헌수': 1, '대행': 1, '상왕': 1, '재궁': 1, '천': 1, '수사': 1, '앞뜰': 1, '머물렀다': 1})


##### 불용어를 더 거르기

In [27]:
# preprocess() 불용어를 더 추가
stopword = read_words('../DATA/stopwords.txt')

def preprocess_2(tokens):
    tokens = [token for token in tokens if re.match(r'[가-힣]+', token)]
    tokens = [token for token in tokens if token not in stopword]
    return tokens

tokens = []
for word in words:
    tokens.extend(okt.morphs(word))

tokens = preprocess_2(tokens)
counter = Counter(tokens)

print(counter)

Counter({'임금': 3, '군신': 2, '거': 2, '느리고': 2, '하고': 2, '백관': 2, '하였다': 2, '안장': 2, '갖춘': 2, '말': 2, '드리고': 2, '대': 2, '수박': 2, '전': 2, '인정전': 1, '정례': 1, '행': 1, '조하': 1, '정지': 1, '각도': 1, '전문': 1, '받': 1, '들고': 1, '칭하': 1, '끝나매': 1, '방물': 1, '바치': 1, '고': 1, '공비': 1, '드렸다': 1, '어소': 1, '나아가': 1, '하례': 1, '안팎': 1, '옷감': 1, '대비': 1, '올렸다': 1, '내전': 1, '들어가': 1, '헌수': 1, '대행': 1, '상왕': 1, '재궁': 1, '천': 1, '수사': 1, '앞뜰': 1, '머물렀다': 1})


### 일단 여기까지, P03 파일로 이어짐